## Install invana-connector

In [1]:
# for stable version 
# pip install invana

# for installation specific branch
#pip install git+https://github.com/invana/invana@refactor/invana-connectors#egg=invana

# for latest code 
import sys
sys.path.append("../")

In [2]:
def print_list(list_data):
    for data in list_data:
        print("",data)

## 1. Modelling graph

### 1.1 Creating graph instance

In [3]:
from invana import InvanaGraph

gremlin_server_url = "ws://megamind.local:8182/gremlin"

graph = InvanaGraph(gremlin_server_url)

[DEBUG:2023-01-22 00:06:37,088:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTING
[DEBUG:2023-01-22 00:06:37,089:connector._init_connection:99] - create driver connection  
[DEBUG:2023-01-22 00:06:38,095:connector.update_connection_state:61] - GraphConnector state updated to : CONNECTED


### 1.2 Defining Nodes and Relationship Models

In [168]:
from invana.ogm.models import VertexModel, EdgeModel
from invana.ogm.fields import StringProperty, DateTimeProperty, IntegerProperty, FloatProperty, BooleanProperty
from invana.ogm import indexes
from datetime import datetime


class Project(VertexModel):
    graph = graph
    properties = {
        'name': StringProperty(max_length=30, trim_whitespaces=True),
        'description': StringProperty(allow_null=True, min_length=5),
        'rating': FloatProperty(allow_null=True),
        'is_active': BooleanProperty(default=True),
        'created_at': DateTimeProperty(default=lambda: datetime.now())
    }
    indexes = (
        indexes.CompositeIndex("name"),
        indexes.CompositeIndex("created_at")
    )


class Person(VertexModel):
    graph = graph
    properties = {
        'first_name': StringProperty(min_length=5, trim_whitespaces=True),
        'last_name': StringProperty(allow_null=True),
        'username': StringProperty(default="rrmerugu"),
        'member_since': IntegerProperty(),

    }
    indexes = (
        indexes.CompositeIndex("username"),
    )


class Authored(EdgeModel):
    graph = graph
    properties = {
        'created_at': DateTimeProperty(default=lambda: datetime.now()),
        'since': IntegerProperty()
    }
    indexes = (
        indexes.CompositeIndex("created_at")
    )

### 1.3 Installing models on graph

In [170]:
from invana.helpers.helpers import install_models

is_install_models = True

if is_install_models:
    # i_understand_rollback=True will roll back any transactions in pending.
    install_models(graph, Project, Person, Authored, i_understand_rollback=True)
else:
    print(f"Ignoring models installation because install_models={install_models}")

[DEBUG:2023-01-22 01:39:30,024:events.log_event:55] - Request a8624281-3034-4f5b-9fa6-e75eb9f1abe4 STARTED with query: mgmt = graph.openManagement()
if (mgmt.containsVertexLabel('Project'))
    Project = mgmt.getVertexLabel('Project')
else 
    Project = mgmt.makeVertexLabel('Project').make()
if (mgmt.containsRelationType('name'))
    name = mgmt.getPropertyKey('name')
else 
    name = mgmt.makePropertyKey('name').dataType(String.class).make()
if (mgmt.containsRelationType('description'))
    description = mgmt.getPropertyKey('description')
else 
    description = mgmt.makePropertyKey('description').dataType(String.class).make()
if (mgmt.containsRelationType('rating'))
    rating = mgmt.getPropertyKey('rating')
else 
    rating = mgmt.makePropertyKey('rating').dataType(Float.class).make()
if (mgmt.containsRelationType('is_active'))
    is_active = mgmt.getPropertyKey('is_active')
else 
    is_active = mgmt.makePropertyKey('is_active').dataType(Boolean.class).make()
if (mgmt.containsRel

[DEBUG:2023-01-22 01:39:30,101:events.log_event:111] - Request 0c22f746-c2bd-419d-8c89-609f0d465936 RESPONSE_RECEIVED:SUCCESS with status code: 200 at 2023-01-22 01:39:30.101729; took -0.000122
[DEBUG:2023-01-22 01:39:30,102:events.log_event:70] - Request 0c22f746-c2bd-419d-8c89-609f0d465936 FINISHED successfully at 2023-01-22 01:39:30.102466; elapsed_time -1.4e-05
[ERROR:2023-01-22 01:39:30,105:events.log_event:126] - Request 574716e2-a0e6-446e-b5f8-f71a8639e99a RESPONSE_RECEIVED:FAILED with error 597: An index with name 'CompositeIndexByCreated_at' has already been defined at 2023-01-22 01:39:30.105557; took -1.6e-05
[ERROR:2023-01-22 01:39:30,105:events.log_event:88] - Request 574716e2-a0e6-446e-b5f8-f71a8639e99a FINISHED with status code : 597:SERVER ERROR EVALUATION at 2023-01-22 01:39:30.105927; elapsed_time -2.3e-05
[DEBUG:2023-01-22 01:39:30,106:events.log_event:55] - Request 951bd87c-de26-4204-b8c4-15d9285cd018 STARTED with query: 
size = graph.getOpenTransactions().size()
if(

Rolling back open transactions...
Rolling back open transactions...
Rolling back open transactions...


TypeError: 'CompositeIndex' object is not iterable

### 1.4 Execute raw query

In [114]:
graph.execute_query("g.V().addV('User').property('name','Ravi')", timeout=180)


[DEBUG:2023-01-22 01:08:28,015:events.log_event:55] - Request 6fe6c9a4-c4b7-472b-b145-eaf073cf0231 STARTED with query: g.V().addV('User').property('name','Ravi');; request_options: {};; at 2023-01-22 01:08:28.015699
[DEBUG:2023-01-22 01:08:28,122:events.log_event:111] - Request 6fe6c9a4-c4b7-472b-b145-eaf073cf0231 RESPONSE_RECEIVED:SUCCESS with status code: 200 at 2023-01-22 01:08:28.121967; took -0.000546
[DEBUG:2023-01-22 01:08:28,127:events.log_event:70] - Request 6fe6c9a4-c4b7-472b-b145-eaf073cf0231 FINISHED successfully at 2023-01-22 01:08:28.127087; elapsed_time -0.000243


<Response:6fe6c9a4-c4b7-472b-b145-eaf073cf0231 status_code=200>

In [115]:
response = graph.execute_query("g.V().limit(1).elementMap().toList()", timeout=180)
print(response)

[DEBUG:2023-01-22 01:08:32,356:events.log_event:55] - Request f773f9ba-3d8c-47a6-97b0-ff71223650e9 STARTED with query: g.V().limit(1).elementMap().toList();; request_options: {};; at 2023-01-22 01:08:32.356190
[DEBUG:2023-01-22 01:08:32,370:events.log_event:111] - Request f773f9ba-3d8c-47a6-97b0-ff71223650e9 RESPONSE_RECEIVED:SUCCESS with status code: 200 at 2023-01-22 01:08:32.370648; took -5.4e-05


<Response:f773f9ba-3d8c-47a6-97b0-ff71223650e9 status_code=200>


[DEBUG:2023-01-22 01:08:32,374:events.log_event:70] - Request f773f9ba-3d8c-47a6-97b0-ff71223650e9 FINISHED successfully at 2023-01-22 01:08:32.374473; elapsed_time -4.3e-05


In [116]:
print(response.data)

[<Node:Organisation id="2113544" name=invana >]


In [117]:
print([d.to_json() for d in response.data])

[{'id': 2113544, 'type': 'vertex', 'label': 'Organisation', 'properties': {'name': 'invana'}}]


## 2 CRUD Operations on data

### 2.1 Nodes

#### 2.1.1 Create Nodes 

In [7]:
person_ravi = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
person_invana = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)


project_iengine = Project.objects.create(name="invana engine", rating=5, is_active=True)
project_iconnector = Project.objects.create(name="invana connectors", rating=5, is_active=True)
project_istudio = Project.objects.create(name="invana studio", rating=5, is_active=True)
project_bengine = Project.objects.create(name="browser engine", rating=5, is_active=False)
# print("Created project_iengine", project_iengine)
# print("Created project_iconnector", project_iconnector)
# print("Created project_istudio", project_istudio)
# print("Created project_bengine", project_bengine)
# print("JSON data", project_iengine.to_json())

#### 2.1.2 Read/Search Nodes

In [62]:
# search
project_search = Project.objects.search().to_list()


## search by ids 
ids = [p.id for p in project_search] 
project_search_has__id = Project.objects.search(has__id=project_search[0].id).to_list()
project_search_has__id_within = Project.objects.search(has__id__within=ids[:2]).to_list()


# search string - eq, neq, startingWith, endingWith, containing, notStartingWith, notEndingWith, notContaining
Project.objects.search(has__name="invana studio").to_list()
Project.objects.search(has__name__eq="invana").to_list()
Project.objects.search(has__name__neq="invana").to_list()
Project.objects.search(has__name__startingWith="invana").to_list()
Project.objects.search(has__name__endingWith="engine").to_list()
Project.objects.search(has__name__containing="eng").to_list()
Project.objects.search(has__name__notStartingWith="invana").to_list()
Project.objects.search(has__name__notEndingWith="engine").to_list()
Project.objects.search(has__name__notContaining="ravi").to_list()


# search int- lt, gt, lte, gte, inside, outside, between
Person.objects.search(has__member_since__lte=3000).to_list()
Person.objects.search(has__member_since__lt=3000).to_list()
Person.objects.search(has__member_since__gte=1999).to_list()
Person.objects.search(has__member_since__gt=1999).to_list()
inside_query = Person.objects.search(has__member_since__inside=(1000, 3000)).to_list()
outside_quuery = Person.objects.search(has__member_since__outside=(1000, 2005)).to_list()
between_query = Person.objects.search(has__member_since__between=(1000, 3000)).to_list()



[<Node:Project id="2904072" name=invana engine created_at=2023-01-21 23:41:56.360000 rating=5.0 is_active=True >,
 <Node:Project id="856192" name=invana connectors created_at=2023-01-21 23:49:04.367000 rating=5.0 is_active=True >,
 <Node:Project id="860288" name=invana engine created_at=2023-01-22 00:06:48.190000 rating=5.0 is_active=True >,
 <Node:Project id="905408" name=invana connectors created_at=2023-01-22 00:06:53.243000 rating=5.0 is_active=True >,
 <Node:Project id="1757400" name=invana connectors created_at=2023-01-21 23:41:56.393000 rating=5.0 is_active=True >,
 <Node:Project id="1765592" name=invana engine created_at=2023-01-21 23:48:59.312000 rating=5.0 is_active=True >]

In [140]:
# order by 
Project.objects.search().order_by('name').to_list()  # asc order
# Project.objects.search().order_by('-name').to_list()  # desc order <--- this is not working 

[<Node:Project id="856192" name=invana connectors created_at=2023-01-21 23:49:04.367000 rating=5.0 is_active=True >,
 <Node:Project id="905408" name=invana connectors created_at=2023-01-22 00:06:53.243000 rating=5.0 is_active=True >,
 <Node:Project id="1757400" name=invana connectors created_at=2023-01-21 23:41:56.393000 rating=5.0 is_active=True >,
 <Node:Project id="2904072" name=invana engine created_at=2023-01-21 23:41:56.360000 rating=5.0 is_active=True >,
 <Node:Project id="860288" name=invana engine created_at=2023-01-22 00:06:48.190000 rating=5.0 is_active=True >,
 <Node:Project id="1765592" name=invana engine created_at=2023-01-21 23:48:59.312000 rating=5.0 is_active=True >]

In [81]:
# using range for pagination
Project.objects.search().order_by('name').range(1, 3).to_list()

[<Node:Project id="905408" name=invana connectors created_at=2023-01-22 00:06:53.243000 rating=5.0 is_active=True >,
 <Node:Project id="1757400" name=invana connectors created_at=2023-01-21 23:41:56.393000 rating=5.0 is_active=True >]

In [124]:
from invana.gremlin.paginator import GremlinQuerySetPaginator
import math 

page_size = 5
total_count = Project.objects.search().count()
total_pages = math.ceil(total_count/page_size)
print("total projects count", total_count)
print("page_size", page_size)
print("total_pages", total_pages)

# create queryset and paginator 
queryset = Project.objects.search().order_by("-name")
paginator = GremlinQuerySetPaginator(queryset, page_size)

# run paginator queries
for page_no in range(1, total_pages+1):
    qs = paginator.page(page_no)
    result = qs.to_list()
    print(f"===== Page {page_no} =====", result)

total projects count 6
page_size 5
total_pages 2
===== Page 1 ===== [<Node:Project id="2904072" name=invana engine created_at=2023-01-21 23:41:56.360000 rating=5.0 is_active=True >, <Node:Project id="856192" name=invana connectors created_at=2023-01-21 23:49:04.367000 rating=5.0 is_active=True >, <Node:Project id="860288" name=invana engine created_at=2023-01-22 00:06:48.190000 rating=5.0 is_active=True >, <Node:Project id="905408" name=invana connectors created_at=2023-01-22 00:06:53.243000 rating=5.0 is_active=True >, <Node:Project id="1757400" name=invana connectors created_at=2023-01-21 23:41:56.393000 rating=5.0 is_active=True >]
===== Page 2 ===== [<Node:Project id="1765592" name=invana engine created_at=2023-01-21 23:48:59.312000 rating=5.0 is_active=True >]


##### 2.1.3 Update Nodes

In [58]:
Project.objects.search(has__name="invana studio").update(name="Invana Studio")

[]

##### 2.1.4 Delete Nodes 

In [61]:
Project.objects.delete(has__name__startingWith="browser") # kwargs are same as in search 
Project.objects.search().to_list()

[<Node:Project id="2904072" name=invana engine created_at=2023-01-21 23:41:56.360000 rating=5.0 is_active=True >,
 <Node:Project id="856192" name=invana connectors created_at=2023-01-21 23:49:04.367000 rating=5.0 is_active=True >,
 <Node:Project id="860288" name=invana engine created_at=2023-01-22 00:06:48.190000 rating=5.0 is_active=True >,
 <Node:Project id="905408" name=invana connectors created_at=2023-01-22 00:06:53.243000 rating=5.0 is_active=True >,
 <Node:Project id="1757400" name=invana connectors created_at=2023-01-21 23:41:56.393000 rating=5.0 is_active=True >,
 <Node:Project id="1765592" name=invana engine created_at=2023-01-21 23:48:59.312000 rating=5.0 is_active=True >]

##### 2.1.1.5 get_or_create

In [54]:
person = Person.objects.get_or_create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
print(person)
person = Person.objects.get_or_create(first_name="Invana", last_name="Technologies", member_since=2016)
print(person)

(False, <Node:Person id="909512" last_name=Merugu first_name=Ravi Raja username=rrmerugu member_since=2000 >)
(False, <Node:Person id="913648" last_name=Technologies first_name=Invana username=rrmerugu member_since=2016 >)


##### 2.1.1.6 get_or_none

In [147]:
Person.objects.get_or_none(first_name="Ravi Raja", last_name="Merugu", member_since=2000)

<Node:Person id="909512" last_name=Merugu first_name=Ravi Raja username=rrmerugu member_since=2000 >

### 2.2 Relationships

#### 2.2.1 Create Relationship

In [174]:
person = Person.objects.create(first_name="Ravi Raja", last_name="Merugu", member_since=2000)
project_iengine = Project.objects.create(name="invana engine", rating=5, is_active=True)
project_iconnector = Project.objects.create(name="invana connectors", rating=5, is_active=True)

authored_data = Authored.objects.create(person.id, project_iengine.id, since=2010)
authored_data2 = Authored.objects.create(person.id, project_iconnector.id, since=2020)

#### 2.2.2 Read/Search Relationship

In [175]:
Authored.objects.search().to_list()
# [d.to_json() for d in data]

[<RelationShip:authored id="u4u9-1ztag-8ph-lif4" 3350536:Person -> authored -> 1003648:Project created_at=2023-01-22 01:28:29.170000 >,
 <RelationShip:authored id="u4g1-1ztag-8ph-mttc" 3350536:Person -> authored -> 1065072:Project created_at=2023-01-22 01:28:29.159000 >,
 <RelationShip:authored id="ucch-1zzm0-8ph-18v6g" 3358728:Person -> authored -> 2093272:Project created_at=2023-01-22 01:41:24.596000 since=2010 >,
 <RelationShip:authored id="9sem-n04w-8ph-m7rc" 1073264:Person -> authored -> 1036488:Project created_at=2023-01-22 01:41:34.413000 since=2010 >,
 <RelationShip:authored id="9ssu-n04w-8ph-18yc8" 1073264:Person -> authored -> 2097368:Project created_at=2023-01-22 01:41:34.423000 since=2020 >,
 <RelationShip:authored id="j9cr-18ouw-8ph-mx2o" 2085080:Person -> authored -> 1069296:Project created_at=2023-01-22 01:23:26.952000 >,
 <RelationShip:authored id="j8yj-18ouw-8ph-18s0o" 2085080:Person -> authored -> 2089176:Project created_at=2023-01-22 01:23:26.944000 >]

In [176]:
Authored.objects.search(has__since__lt=2020).to_list() # refer search_kwargs from Node search section(2.1.2)

[<RelationShip:authored id="ucch-1zzm0-8ph-18v6g" 3358728:Person -> authored -> 2093272:Project since=2010 created_at=2023-01-22 01:41:24.596000 >,
 <RelationShip:authored id="9sem-n04w-8ph-m7rc" 1073264:Person -> authored -> 1036488:Project since=2010 created_at=2023-01-22 01:41:34.413000 >]

#### 2.2.3 Update Relationship

In [180]:
Authored.objects.search(has__since__lt=2020).update(since=2050)


[]

#### 2.2.4 Delete Relationship


In [182]:
Authored.objects.delete()

[['E'], ['hasLabel', 'authored'], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]